# Excel to SQL Server

This script creates a table and inserts values from Excel file into SQL Server

In [ ]:
import pandas.io.sql
import pyodbc
import pandas as pd
import datetime
import xlrd

In [ ]:
server = '[SERVER NAME]'
db = '[DATABASE NAME]'

# create Connection and Cursor objects
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_Connection=yes')
cursor = conn.cursor()

If you need to rename your columns for database, the code below does right that

In [ ]:
# read data
#data = pd.read_excel('covid1.xls')
'''
# rename columns
data = data.rename(columns={'date Rep': 'dateRep',
                            'continent Exp': 'continentExp'})
'''
# export
#data.to_excel('covid1.xls', index=False)

In [ ]:
# Open the workbook and define the worksheet
book = xlrd.open_workbook("covid1.xls")
sheet = book.sheet_by_name("Sheet1")

Create table query with rows from Excel file

In [ ]:
query1 = """
DROP TABLE IF EXISTS covidn;
CREATE TABLE covidn (
    dateRep nchar(25),
    day int,
    month int,
    year int,
    cases int,
    deaths nchar(100),
    countriesAndTerritories nchar(100),
    geold nchar(100),
    countryterritoryCode nchar(100),
    popData2019 int,
    continentExp nchar(10)
)"""

Insert values into table template

In [ ]:
query = """
INSERT INTO covidn (
    dateRep,
    day,
    month,
    year,
    cases,
    deaths,
    countriesAndTerritories,
    geold,
    countryterritoryCode,
    popData2019,
    continentExp) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

In [ ]:
# execute create table
try:
    cursor.execute(query1)
    conn.commit()
except pyodbc.ProgrammingError:
    pass

In [ ]:
# takes existing row count in the database for validation later
cursor.execute("SELECT count(*) FROM covidn")
before_import = cursor.fetchone()

In [ ]:
for r in range(1, sheet.nrows):
    dateRep = sheet.cell(r,0).value
    day = int(sheet.cell(r,1).value)
    month = int(sheet.cell(r,2).value)
    year = int(sheet.cell(r,3).value)
    cases = int(sheet.cell(r,4).value)
    deaths = int(sheet.cell(r,5).value)
    countriesAndTerritories = sheet.cell(r,6).value
    geold = sheet.cell(r,7).value
    countryterritoryCode = sheet.cell(r,8).value
    try:
        popData2019 = int(sheet.cell(r,9).value)
    except:
        popData2019 = 0
    continentExp = sheet.cell(r,10).value

    # Assign values from each row
    values = (dateRep, day, month, year, cases, deaths, countriesAndTerritories, geold, 
              countryterritoryCode, popData2019, continentExp)

    # Execute sql query
    cursor.execute(query, values)

# Commit the transaction
conn.commit()

In [ ]:
# If you want to check if all rows are imported
cursor.execute("SELECT count(*) FROM covidn")
result = cursor.fetchone()

print((result[0] - before_import[0]) == len(data.index))  # should be True in case all rows were imported to database

# Close the database connection
conn.close()